Cat or Dog Neural Network

In [78]:
#load modules
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import preprocessing
import tensorflow as tf

Image Processing

In [93]:
#goes through each image and checks if it is non-corrupt, deletes if it is corrupted
import os
from PIL import Image
folder_path = 'PetImages'
extensions = []
for fldr in os.listdir(folder_path):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in os.listdir(sub_folder_path):
        file_path = os.path.join(sub_folder_path, filee)
        print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        try:
            im = Image.open(file_path)
            rgb_im = im.convert('RGB')
            if filee.split('.')[1] not in extensions:
                extensions.append(filee.split('.')[1])
        except: 
            os.remove(file_path)

c:\Users\Wiggles\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:819: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Using the above section of code, cats/666.jpg and dogs/11702.jpg were identified as corrupt and deleted

In [94]:
training_data_generator = ImageDataGenerator(
        #Rescale the image by 1/255 to normalize the pixel values
        rescale=1.0/255,
        #Randomly increase or decrease the size of the image by up to 20%
        zoom_range=0.2, 
        #Randomly rotate the image between -15,15 degrees
        rotation_range=15, 
        #Shift the image along its width by up to +/- 5%
        width_shift_range=0.05, 
        #Shift the image along its height by up to +/- 5%
        height_shift_range=0.05 
)

In [95]:
#variables being fed to training data generator
directory = "PetImages" #path to the folder containing the images
class_mode = "categorical"
color_mode = "rgb" 
target_size = (256, 256) #resizes each image to 256x256
batch_size = 16

In [96]:
training_iterator = training_data_generator.flow_from_directory(directory, class_mode =class_mode ,color_mode =color_mode ,target_size = target_size, batch_size = batch_size)

Found 24998 images belonging to 2 classes.


Mode Building

In [97]:
model = tf.keras.Sequential()

In [99]:
model.add(tf.keras.Input(shape=(256, 256, 3)))
model.add(tf.keras.layers.Conv2D(2, 5, strides=3, activation="relu")) 
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(5, 5), strides=(5,5)))
model.add(tf.keras.layers.Conv2D(4, 3, strides=1, activation="relu")) 
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2,2), strides=(2,2)))
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(2,activation="softmax"))

In [100]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 196608)            0         
                                                                 
 dense_15 (Dense)            (None, 100)               19660900  
                                                                 
 dense_16 (Dense)            (None, 50)                5050      
                                                                 
 dense_17 (Dense)            (None, 2)                 102       
                                                                 
Total params: 19,666,052
Trainable params: 19,666,052
Non-trainable params: 0
_________________________________________________________________


In [101]:
#model compilation
#model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC()])

In [102]:
validation_data_generator = ImageDataGenerator( rescale=1./255)
validation_iterator = validation_data_generator.flow_from_directory(directory, class_mode =class_mode ,color_mode =color_mode ,target_size = target_size, batch_size = batch_size)

Found 24998 images belonging to 2 classes.


In [103]:
model.fit(
       training_iterator,
       steps_per_epoch=training_iterator.samples/batch_size,
       epochs=5,
       validation_data=validation_iterator,
       validation_steps=validation_iterator.samples/batch_size)

Epoch 1/5
1562/1562 [==============================] - 434s 278ms/step - loss: 3.0027 - categorical_accuracy: 0.5131 - auc_4: 0.5157 - val_loss: 0.6932 - val_categorical_accuracy: 0.5000 - val_auc_4: 0.5000
Epoch 2/5
1562/1562 [==============================] - 432s 276ms/step - loss: 0.6934 - categorical_accuracy: 0.5052 - auc_4: 0.5037 - val_loss: 0.6937 - val_categorical_accuracy: 0.5000 - val_auc_4: 0.5000
Epoch 3/5
1562/1562 [==============================] - 425s 272ms/step - loss: 0.6935 - categorical_accuracy: 0.5002 - auc_4: 0.4996 - val_loss: 0.6932 - val_categorical_accuracy: 0.5000 - val_auc_4: 0.5000
Epoch 4/5
1562/1562 [==============================] - 423s 271ms/step - loss: 0.6935 - categorical_accuracy: 0.4966 - auc_4: 0.4966 - val_loss: 0.6946 - val_categorical_accuracy: 0.5000 - val_auc_4: 0.5000
Epoch 5/5
1562/1562 [==============================] - 424s 271ms/step - loss: 0.6935 - categorical_accuracy: 0.4956 - auc_4: 0.4973 - val_loss: 0.6932 - val_categorical_ac